In [1]:
import mysql.connector as sql
import pandas as pd
import datetime as dt
import numpy as np
from sorted_months_weekdays import *
from sort_dataframeby_monthorweek import *
import matplotlib.pyplot as plt
import math
import json
from pandas.io.json import json_normalize

In [2]:
db_connection = sql.connect(host='localhost', database='collection_visits', user='pandas', password='password')

visits = pd.read_sql('SELECT * FROM Visits', con=db_connection)
visits['created'] = pd.to_datetime(visits['created'], unit='s')
visits['lastmodified'] = pd.to_datetime(visits['lastmodified'], unit='s')
visits['registered'] = visits['registered'].replace(0, np.nan)
visits['registered'] = pd.to_datetime(visits['registered'], unit='s')
visits['deleted'] = visits['deleted'].replace(0, np.nan)
visits['deleted'] = pd.to_datetime(visits['deleted'], unit='s')
visits['paired'] = visits['paired'].replace(0, np.nan)
visits['paired'] = pd.to_datetime(visits['paired'], unit='s')
visits['created_ip'].replace(0, np.nan, inplace=True)
visits['registered_ip'].replace(0, np.nan, inplace=True)
visits['paired_ip'].replace(0, np.nan, inplace=True)
# visits['processed'].replace(0, np.nan, inplace=True)
visits['exported'].replace(0, np.nan, inplace=True)
visits['title'].fillna(value=np.nan, inplace=True)
visits['note'].replace(r'^\s*$', np.NaN, regex=True, inplace=True)

In [3]:
db_connection = sql.connect(host='localhost', database='collection_users', user='pandas', password='password')

sbBundles = pd.read_sql('SELECT * FROM UsersShoeboxBundles', con=db_connection)
sbBundles['created'] = pd.to_datetime(sbBundles['created'], unit='s')
sbBundles['lastmodified'] = pd.to_datetime(sbBundles['lastmodified'], unit='s')
sbBundles['deleted'].replace(0, np.nan, inplace=True)
sbBundles['deleted'] = pd.to_datetime(sbBundles['deleted'], unit='s')
sbBundles['description'].replace(r'^\s*$', np.NaN, regex=True, inplace=True)

In [15]:
visits_created = visits[['user_id','created']]
visits_created

,user_id,created
0,0,2014-10-29 21:10:47
1,0,2014-10-29 23:32:29
2,0,2014-10-29 23:40:51
3,0,2014-10-29 23:43:31
4,0,2014-10-30 00:13:47
...,...,...
716277,0,2020-07-09 13:11:50
716278,0,2020-07-09 13:16:19
716279,0,2020-07-09 13:19:41
716280,0,2020-07-09 13:40:43


In [17]:
sbBundles_created = sbBundles[['user_id', 'created', 'count_items']]
sbBundles_created = sbBundles_created.rename({'created': 'accessed'}, axis=1)

In [19]:
sbBundles_created

,user_id,accessed,count_items
0,199,2014-09-30 17:24:19,3
1,231,2014-10-02 19:59:12,5
2,247,2014-10-06 19:34:33,0
3,247,2014-10-07 21:38:07,0
4,247,2014-11-03 21:32:25,0
...,...,...,...
23121,28814,2020-04-22 20:58:47,3
23122,28814,2020-04-22 22:01:58,4
23123,28814,2020-07-02 17:21:25,1
23124,28814,2020-07-22 20:26:22,5


In [20]:
visit_bundle = pd.merge(visits_created, sbBundles_created, on='user_id')

In [22]:
visit_bundle['time_between'] = visit_bundle['accessed'] - visit_bundle['created']

In [30]:

#i need to choose the first created and the first accesse

,created,accessed,count_items,time_between
user_id,,,,
247,2014-11-03 21:13:10,2014-10-06 19:34:33,0,-29 days +22:21:23
247,2014-11-03 21:13:10,2014-10-07 21:38:07,0,-27 days +00:24:57
247,2014-11-03 21:13:10,2014-11-03 21:32:25,0,0 days 00:19:15
247,2014-11-03 21:13:10,2014-11-05 23:16:31,0,2 days 02:03:21
247,2014-11-03 21:13:10,2015-01-06 21:29:28,0,64 days 00:16:18
...,...,...,...,...
28814,2020-07-06 17:41:57,2020-04-22 20:58:47,3,-75 days +03:16:50
28814,2020-07-06 17:41:57,2020-04-22 22:01:58,4,-75 days +04:20:01
28814,2020-07-06 17:41:57,2020-07-02 17:21:25,1,-5 days +23:39:28


In [53]:
first_time_visit = visit_bundle.groupby(by='user_id').created.first()
first_time_visit = first_time_visit.to_frame().reset_index()    

In [54]:
first_time_accessed = visit_bundle.groupby(by='user_id').accessed.first()
first_time_accessed = first_time_accessed.to_frame().reset_index()

In [57]:
visit_bundle_first_times = pd.merge(first_time_visit, first_time_accessed, on='user_id')
visit_bundle_first_times

,user_id,created,accessed
0,3,2015-02-24 18:29:06,2015-02-24 18:29:10
1,18,2015-05-22 18:40:19,2015-05-22 19:04:45
2,25,2015-09-22 16:29:05,2015-09-22 20:20:30
3,94,2015-07-03 15:07:46,2019-03-26 17:47:51
4,105,2016-06-23 14:44:59,2016-06-23 20:43:39
...,...,...,...
21053,30935,2020-02-23 18:52:03,2020-07-18 14:19:38
21054,30957,2020-01-03 20:10:11,2020-07-27 22:02:18
21055,30959,2020-01-16 14:50:56,2020-07-28 20:44:05
21056,30980,2020-03-01 21:17:53,2020-08-11 02:16:51


In [58]:
visit_bundle_first_times['time_between'] = visit_bundle_first_times['accessed'] - visit_bundle_first_times['created']

In [59]:
visit_bundle_first_times

,user_id,created,accessed,time_between
0,3,2015-02-24 18:29:06,2015-02-24 18:29:10,0 days 00:00:04
1,18,2015-05-22 18:40:19,2015-05-22 19:04:45,0 days 00:24:26
2,25,2015-09-22 16:29:05,2015-09-22 20:20:30,0 days 03:51:25
3,94,2015-07-03 15:07:46,2019-03-26 17:47:51,1362 days 02:40:05
4,105,2016-06-23 14:44:59,2016-06-23 20:43:39,0 days 05:58:40
...,...,...,...,...
21053,30935,2020-02-23 18:52:03,2020-07-18 14:19:38,145 days 19:27:35
21054,30957,2020-01-03 20:10:11,2020-07-27 22:02:18,206 days 01:52:07
21055,30959,2020-01-16 14:50:56,2020-07-28 20:44:05,194 days 05:53:09
21056,30980,2020-03-01 21:17:53,2020-08-11 02:16:51,162 days 04:58:58


In [62]:
visit_bundle_first_times['time_between'].mean()

Timedelta('12 days 07:09:08.464763')